## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice


def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [11]:
import networkx as nx
import numpy as np

In [29]:
def solve_tsp_nearest_neighbour(instance):
    n = len(instance)
    candidates = {i for i in range(1, n)}
    permutation = [0]
    n = len(instance)
    for i in range(n - 1):
        tmp = list(candidates)
        next_v = np.argmin([euclidean_distance(instance[permutation[i]], instance[j]) for j in tmp])
        permutation.append(tmp[next_v])
        candidates.remove(tmp[next_v])
    return permutation

In [30]:
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [31]:
def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    # Replace this trivial lower bound with Held—Karp:
    vertex = len(vertex_coordinates)
    graph = nx.Graph()
    for i in range(1, vertex):
        for j in range(i + 1, vertex):
            graph.add_edge(i, j, weight=euclidean_distance(vertex_coordinates[i], vertex_coordinates[j]))
    y = np.zeros(vertex)
    dy = np.zeros(vertex)
    alpha = 2.0
    beta = 0.5
    num_iters = 20
    U = calculate_tour_length(vertex_coordinates, solve_tsp_nearest_neighbour(vertex_coordinates))
    weights = [euclidean_distance(vertex_coordinates[0], vertex_coordinates[i]) for i in range(1, vertex)]
    weights.sort()
    A = weights[0] + weights[1]
    H = -1000000000000
    TSMALL = 0.001
    for iteration in range(num_iters):
        T = nx.minimum_spanning_tree(graph)
        degree = np.zeros(vertex)
        H_1 = 2 * y.sum() + A + T.size(weight='weight')
        if H_1 > H:
            H = H_1
        for edge in T.edges(data=True):
            degree[edge[0]] += 1
            degree[edge[1]] += 1
        t_k = alpha * (U - H_1) / ((2 - degree) ** 2).sum()
        if t_k < TSMALL:
            break
        dy = t_k * (2 - degree)
        y += dy
        
        for edge in graph.edges(data=True):
            edge[-1]['weight'] -= dy[edge[0]]
            edge[-1]['weight'] -= dy[edge[1]]
        alpha *= beta
    return H

In [32]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [33]:
run_all()

Instance pr107.tsp… done in 0.43 seconds with lower bound 36923
Instance pr152.tsp… done in 0.94 seconds with lower bound 59772
Instance d198.tsp… done in 1.6 seconds with lower bound 12944
Instance pr439.tsp… done in 1e+01 seconds with lower bound 92443
Instance d493.tsp… done in 1.2e+01 seconds with lower bound 31292
Instance d657.tsp… done in 2.4e+01 seconds with lower bound 43858
Instance d2103.tsp… done in 3.1e+02 seconds with lower bound 77433


## Выводы
Запишите здесь полученные результаты относительно погрешностей алгоритмов NN и NI.

Solving instance d198.tsp… done in 0.013 seconds with tour length 18620 using NN and in 0.91 seconds with tour length 18050 using NI
Solving instance d493.tsp… done in 0.07 seconds with tour length 43646 using NN and in 1.2e+01 seconds with tour length 42130 using NI
Solving instance d657.tsp… done in 0.13 seconds with tour length 62176 using NN and in 2.7e+01 seconds with tour length 60081 using NI
Solving instance d2103.tsp… done in 1.2 seconds with tour length 87468 using NN and in 8.7e+02 seconds with tour length 85771 using NI
Solving instance pr107.tsp… done in 0.0042 seconds with tour length 46678 using NN and in 0.19 seconds with tour length 53211 using NI
Solving instance pr152.tsp… done in 0.0076 seconds with tour length 85702 using NN and in 0.48 seconds with tour length 88661 using NI
Solving instance pr439.tsp… done in 0.061 seconds with tour length 131282 using NN and in 1.2e+01 seconds with tour length 133705 using NI

In [35]:
best_ans = np.array([18620, 43646, 62176, 87468, 53211, 88661, 133705])
bound = np.array([12944, 31292, 43858, 77433, 36923, 59772, 92443])
best_ans / bound

array([ 1.43850433,  1.39479739,  1.4176661 ,  1.12959591,  1.44113425,
        1.48331995,  1.44635072])

Как видим NN и NI решают задачу не более чем в 1.5 раза хуже чем нижняя оценка. Значит они решают достаточно хорошо.